In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%env WANDB_DISABLED=True

In [ ]:
import pandas as pd
dataset = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
dataset.head()

In [ ]:
df = dataset.drop(['keyword', 'location'], axis=1)
df.head()

In [ ]:
df.isna().sum()

In [ ]:
len(df[df['target']==1])

In [ ]:
len(df[df['target']==0])

In [ ]:
df['text']=df['text'].apply(lambda x: x.lower())

In [ ]:
df['text']=df['text'].apply(lambda x: x.replace("#", ""))

In [ ]:
import string 
print(string.punctuation)

In [ ]:
#str.maketrans(dict.fromkeys(string.punctuation))

In [ ]:
df['text']=df['text'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation))))

In [ ]:
df['text'][5]

In [ ]:
df['text']=df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))

In [ ]:
df['text'][5]

In [ ]:
for index, row in df.iterrows():
    text=[]
    for word in row['text'].split():
        if "http" not in word:
            text.append(word)
    row['text']=" ".join(text)
    #print(row['text'])
    df['text'][index]=row['text']
        

In [ ]:
#!wget https://raw.githubusercontent.com/vishalrk1/pytorch/main/nlp_helper.py

In [ ]:
#from nlp_helper import remove_html, remove_punctuation, lowercase_text, word_lemmatizer
import nltk
nltk.download('wordnet')

# Remove punctuation
"""def remove_punctuation(text):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ''.join([c for c in text if c not in punc])
    no_punct = no_punct.lower()
    return no_punct

def preprocess_text(text):
    text = remove_html(text)
    text = remove_punctuation(text)
      # text = remove_stopwords(text)
      # text = lowercase_text(text)
    text = word_lemmatizer(text)
    text = ''.join(text)
    return text
df['text']=df['text'].apply(lambda x:preprocess_text(x))"""

In [ ]:
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text
df['text']=df['text'].apply(lambda x:''.join(word_lemmatizer(x)))

In [ ]:
df['text'][7612]

In [ ]:
!pip install transformers

In [ ]:
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, default_data_collator, Trainer, set_seed,AutoTokenizer, AutoModelForSequenceClassification
set_seed(42)
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

In [ ]:
maxx, minn = 0, 1000000
for index, row in df.iterrows():
    if len(row['text'].split())>maxx:
        maxx=len(row['text'].split())
    if len(row['text'].split())<minn:
        minn=len(row['text'].split())
maxx, minn

In [ ]:
max_length=64
df['input_ids']=df['text'].apply(lambda x: tokenizer(x, max_length=max_length, padding='max_length')['input_ids'])
df.head(n=10)

In [ ]:
df.rename(columns={'target':'labels'}, inplace=True)
df.head()

In [ ]:
df = df[['input_ids', 'labels']]
df.head()

In [ ]:
train_df = df[:-int(len(df)*0.01)].reset_index(drop=True)
test_df = df[-int(len(df)*0.01):].reset_index(drop=True)

In [ ]:
len(train_df), len(test_df)

In [ ]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [ ]:
train_ds

In [ ]:
batch_size = 16

args = TrainingArguments(
    'nlp-getting-started',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=3e-5,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

data_collator = default_data_collator
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

In [ ]:
df_submission = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df_submission.head()

In [ ]:
df_submission['text']=df_submission['text'].apply(lambda x: x.lower())

In [ ]:
df_submission['text']=df_submission['text'].apply(lambda x: x.replace("#", ""))

In [ ]:
df_submission['text']=df_submission['text'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation))))

In [ ]:
df_submission['text']=df_submission['text'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))

In [ ]:
for index, row in df_submission.iterrows():
    text=[]
    for word in row['text'].split():
        if "http" not in word:
            text.append(word)
    row['text']=" ".join(text)
    #print(row['text'])
    df_submission['text'][index]=row['text']

In [ ]:
df_submission['text']=df_submission['text'].apply(lambda x:''.join(word_lemmatizer(x)))

In [ ]:
df_submission['text'][89]

In [ ]:
df_submission['input_ids']=df_submission['text'].apply(lambda x: tokenizer(x, max_length=max_length, padding='max_length')['input_ids'])
df_submission.head()

In [ ]:
df_submission=df_submission[['input_ids']]
df_submission.head()

In [ ]:
output_ds = Dataset.from_pandas(df_submission)
output_ds

In [ ]:
outputs=trainer.predict(output_ds)

In [ ]:
outputs[0][0]

In [ ]:
sub = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
sub.head()

In [ ]:
sub['target'] = outputs.predictions.argmax(1)
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)